<a href="https://colab.research.google.com/github/EmersonB/Virginia-Crashes-SVM-Analysis/blob/main/ML_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Loading & Manipulation**

In [ ]:
# Import some common packages
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# Your code goes here for this section, make sure you also include the output to answer the above questions.
import pandas as pd # Pandas module to enable data manipulation

def load_crash_data():
    csv_path = os.path.join("", "Virginia_Crashes.csv")
    return pd.read_csv(csv_path, delimiter=",")
def get_file_in_mb():
    size = os.path.getsize("Virginia_Crashes.csv")
    return str(round(size / (1024 * 1024), 3))

crash2 = load_crash_data()
crash_size = get_file_in_mb()
# Display the first n rows of the data. (n=5 by default, we will use 10 rows)
crash2.head(10)
print(crash_size)

670.131


In [ ]:
crash=crash2.copy()

In [ ]:
crash = crash.head(100000)

crash = crash.infer_objects()
crash = crash.convert_dtypes()
crash['Driverage'].fillna('30', inplace=True)


for i in range(len(crash['Driverage'])):
  age = crash['Driverage'][i]
  if(";" in str(age)):
    temp = age.split(";")
    min_age = min(temp)
    age = min_age
  else:
    age = int(age)
  crash['Driverage'][i] = str(age)

for i in range(len(crash['Drivergen'])):
  gen = crash['Drivergen'][i]
  gen2 = "0"
  if(";" in str(gen)):
    temp = gen.split(";")
    gen = temp[0]
  if(gen == 'Female'):
    gen2 = "1"
  if(gen == 'Not Provided'):
    gen2 = "2"
  crash['Drivergen'][i] = gen2

for i in range(len(crash['Collision_Type'])):
  tp = crash['Collision_Type'][i]
  tp = tp[0]
  #print(tp)
  if not tp.isdigit():
    tp = "0"
  crash['Collision_Type'][i] = tp

count = 0
for i in range(len(crash['Alcohol_Notalcohol'])):
  al = crash['Alcohol_Notalcohol'][i]
  if(al == "ALCOHOL"):
    count+=1
    al = "1"
  else:
    al = "-1"
  crash['Alcohol_Notalcohol'][i] = al

print("count",count)


print(crash['Crash_Severity'].iloc[0:40])

crash['Drivergen'] = crash['Drivergen'].astype(int)


crash['Alcohol_Notalcohol'] = crash['Alcohol_Notalcohol'].astype(int)

crash['Collision_Type'] = crash['Collision_Type'].astype(int)

crash['Driverage'] = crash['Driverage'].astype(int)


crash.info()
print(crash.iloc[0])

count 6526
0             B.Visible Injury
1          C.Nonvisible Injury
2     PDO.Property Damage Only
3     PDO.Property Damage Only
4     PDO.Property Damage Only
5     PDO.Property Damage Only
6     PDO.Property Damage Only
7     PDO.Property Damage Only
8     PDO.Property Damage Only
9     PDO.Property Damage Only
10         C.Nonvisible Injury
11    PDO.Property Damage Only
12    PDO.Property Damage Only
13    PDO.Property Damage Only
14    PDO.Property Damage Only
15             A.Severe Injury
16    PDO.Property Damage Only
17    PDO.Property Damage Only
18            B.Visible Injury
19    PDO.Property Damage Only
20    PDO.Property Damage Only
21            B.Visible Injury
22    PDO.Property Damage Only
23            B.Visible Injury
24    PDO.Property Damage Only
25            B.Visible Injury
26    PDO.Property Damage Only
27    PDO.Property Damage Only
28    PDO.Property Damage Only
29    PDO.Property Damage Only
30    PDO.Property Damage Only
31    PDO.Property Damage On

In [ ]:
print(crash['Collision_Type'].iloc[0:40])

0     9
1     1
2     1
3     9
4     9
5     1
6     2
7     9
8     2
9     1
10    9
11    9
12    9
13    9
14    9
15    9
16    8
17    2
18    9
19    9
20    4
21    1
22    2
23    2
24    9
25    2
26    9
27    2
28    4
29    9
30    1
31    9
32    9
33    1
34    1
35    9
36    1
37    1
38    1
39    2
Name: Collision_Type, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# y = y*2-1.0 # convert the labels from {0,1} to {-1, +1}

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
#                                                     random_state=42)
# print(crash.info)

# crash = crash.drop(['Crash_Dt'], axis=1)
crash = crash[['Alcohol_Notalcohol','Speed_Notspeed', 'Belted_Unbelted', 'Driverage','Drivergen','Crash_Severity','Collision_Type']]

# crash = crash.dropna()
train_set, test_set = train_test_split(crash, test_size=0.2, random_state=42)
X_train = train_set.drop("Alcohol_Notalcohol", axis=1);
y_train = train_set["Alcohol_Notalcohol"].copy();
X_test = test_set.drop("Alcohol_Notalcohol", axis=1);
y_test = test_set["Alcohol_Notalcohol"].copy();


In [ ]:
crash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Alcohol_Notalcohol  100000 non-null  int64 
 1   Speed_Notspeed      100000 non-null  string
 2   Belted_Unbelted     100000 non-null  string
 3   Driverage           100000 non-null  int64 
 4   Drivergen           100000 non-null  int64 
 5   Crash_Severity      100000 non-null  string
 6   Collision_Type      100000 non-null  int64 
dtypes: int64(4), string(3)
memory usage: 5.3 MB


In [ ]:
print(crash.iloc[0])

Speed_Notspeed                         Not_SPEED
Belted_Unbelted                           BELTED
Driverage                                     27
Drivergen                                   Male
Crash_Severity          PDO.Property Damage Only
Collision_Type     4. Sideswipe - Same Direction
Name: 9254, dtype: object


In [ ]:
print(crash['Drivergen'][0:50])

9254      Male
1561      Male
1670    Female
6087      Male
6669    Female
5933      Male
8829    Female
7945    Female
3508      Male
2002    Female
5866    Female
168     Female
2588      Male
5905      Male
9529      Male
6384    Female
7824    Female
9779      Male
4629    Female
8560    Female
9291      Male
6874    Female
6521      Male
8979      Male
6647      Male
9825      Male
210       Male
5278    Female
1919      Male
718       Male
4868    Female
7834    Female
1298      Male
9193    Female
4617      Male
3128    Female
9795    Female
3979    Female
9924    Female
8317      Male
4393      Male
1287      Male
4355    Female
7013      Male
3885    Female
5749      Male
9169      Male
2407      Male
2013    Female
5153      Male
Name: Drivergen, dtype: string


In [ ]:
corr_matrix = crash.corr()
corr_matrix.shape

(1, 1)

In [ ]:
from sklearn.impute import SimpleImputer
# y_train = y_train*2-1.0
# y_test = y_test * 2-1.0

# for col in crash.columns: 
#     print(col) 

crash = X_train.copy()
crash_test = X_test.copy()

print(train_set.shape, test_set.shape)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


crash_num = crash.select_dtypes(include=[np.number])
num_attribs = list(crash_num)
num_attribs = ["Driverage",'Collision_Type','Drivergen']
# cat_attribs = ["Speed_Notspeed","Belted_Unbelted","Driverage","Collision_Type","Time"] #we should use bins for driverage
crash_cat = crash.select_dtypes(include=['string'])
cat_attribs = list(crash_cat)
cat_attribs = ['Speed_Notspeed', 'Belted_Unbelted','Crash_Severity',]

print(num_attribs)
print(cat_attribs)
print(crash.head(10))

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

y_pipeline = ColumnTransformer([
          ("num", num_pipeline, ["Alcohol_Notalcohol"])                       
])

crash_prepared = full_pipeline.fit_transform(crash)
crash_test_prepared = full_pipeline.transform(crash_test)

# y_train = y_pipeline.fit_transform(y_train)
# y_test = y_pipeline.transform(y_test)


# y_test = y_test.replace("ALCOHOL", 1.0)
# y_test = y_test.replace("Not_ALCOHOL",-1.0)

# y_train = y_train.replace("ALCOHOL", 1.0)
# y_train = y_train.replace("Not_ALCOHOL",-1.0)

print(crash_prepared.shape)
print(y_train.shape)



(80000, 7) (20000, 7)
(80000, 6)
(80000,)
(20000, 6)
(20000,)
['Driverage', 'Collision_Type', 'Drivergen']
['Speed_Notspeed', 'Belted_Unbelted', 'Crash_Severity']
      Speed_Notspeed Belted_Unbelted  ...            Crash_Severity  Collision_Type
75220      Not_SPEED          BELTED  ...  PDO.Property Damage Only               1
48955      Not_SPEED          BELTED  ...          B.Visible Injury               1
44966      Not_SPEED          BELTED  ...       C.Nonvisible Injury               1
13568      Not_SPEED          BELTED  ...  PDO.Property Damage Only               2
92727      Not_SPEED          BELTED  ...          B.Visible Injury               1
51349          SPEED          BELTED  ...  PDO.Property Damage Only               1
86979      Not_SPEED          BELTED  ...  PDO.Property Damage Only               1
3806       Not_SPEED        UNBELTED  ...          B.Visible Injury               2
91822      Not_SPEED        UNBELTED  ...          B.Visible Injury              

In [ ]:
print(y_train.iloc[0:50])

75220   -1.0
48955   -1.0
44966   -1.0
13568   -1.0
92727   -1.0
51349   -1.0
86979   -1.0
3806    -1.0
91822   -1.0
6006    -1.0
55086    1.0
44204    1.0
32409   -1.0
88994   -1.0
8138    -1.0
2007    -1.0
89851   -1.0
49900   -1.0
76308   -1.0
37721   -1.0
30391   -1.0
12604    1.0
29365   -1.0
99209   -1.0
34444   -1.0
52417   -1.0
52031   -1.0
47723   -1.0
5612    -1.0
37266   -1.0
74932   -1.0
76079   -1.0
54744   -1.0
98989   -1.0
7576    -1.0
59926   -1.0
46956   -1.0
52932   -1.0
47146   -1.0
41075   -1.0
13175   -1.0
81495   -1.0
5633    -1.0
21915   -1.0
15693   -1.0
57045   -1.0
74184   -1.0
59925   -1.0
67805   -1.0
80164    1.0
Name: Alcohol_Notalcohol, dtype: float64


- - -
## 2. TRAIN SVM FOR CLASSIFICATION TASK 

Use the standard libarary SVM classifier (SVC) on the training data, and then test the classifier on the test data. You will need to call SVM with 3 kernels: (1) Linear, (2) Polynomial and (3) Gaussian RBF. 

You should tune each model using a grid search or similar hyperparameter selection process, and report the best hyperparameters found. You will use these same hyperparameter settings later when testing and comparing to your implementation in Task 4. Once you've selected the best hyperparameters for each kernel, you will need to report the following:

* Confusion matrix, Recall, and Precision. If applicable, discuss any tuning process on C and/or gamma to get to a reasonable result.
* Use the provided plot_svm() to visualize the SVM in 2D. This might give you some insight on how SVM determines the margin and support vector on the Moon dataset.


In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import numpy as np

# Your code here
linear_svc = SVC(kernel = 'linear', C=5, random_state = 42)


# grid_search = GridSearchCV(linear_svc, {'C':[0.0001, 0.1, 0.5,1,2,3,4,5,6,7,8,9]},cv=5, scoring ="recall") 
# grid_search.fit(crash_prepared, y_train)
# print(grid_search.best_params_ )
#The grid search found an optimal C value of 2 for precision and recall

print(crash_prepared.shape)
print(y_train.shape)
print(crash_test_prepared.shape)
print(y_test.shape)
linear_svc.fit(crash_prepared, y_train)
linear_predict = linear_svc.predict(crash_test_prepared)

linear_confusion_matrix = metrics.confusion_matrix(y_test, linear_predict)
print(linear_confusion_matrix) 
#we can see from the confusion matrix that there are minimal false positives and negatives.




(80000, 12)
(80000,)
(20000, 12)
(20000,)
[[18684     0]
 [ 1316     0]]


In [ ]:
metrics.precision_score(y_test, linear_predict)

0.0

In [ ]:
metrics.recall_score(y_test, linear_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import numpy as np

poly_svc = SVC(kernel = 'poly', C=5, gamma = 5, random_state = 42)


# grid_search = GridSearchCV(poly_svc, {'C':[0.2,0.5,1,2,3,4,5,6,7,8,9,10,12],'gamma':[2,4,6,8,10]}, cv=5, scoring ="recall") 
# grid_search.fit(X_train, y_train)
# grid_search.best_params_ 
#The grid search found different values for C and gamma depending on if we want better recall or precision. We will use 0.2 and 2 respectively.


poly_svc.fit(crash_prepared, y_train)
poly_predict = poly_svc.predict(crash_test_prepared)

poly_confusion_matrix = metrics.confusion_matrix(y_test, poly_predict)
print(poly_confusion_matrix) 

# poly_svc.fit(X_train, y_train)
# poly_predict = poly_svc.predict(X_test)
# poly_confusion_matrix = metrics.confusion_matrix(y_test, poly_predict)
# print(poly_confusion_matrix) 
#we can see from the confusion matrix that there are minimal false positives and negatives.

[LibSVM][[178  14]
 [  7   1]]


precision = tp /(tp+fp) = (1/(15)) = 0.0667
recall = tp/(tp+fn) = (1/8) = 0.125

In [ ]:

poly_svc.fit(crash_prepared, y_train)
poly_predict = poly_svc.predict(crash_test_prepared)

poly_confusion_matrix = metrics.confusion_matrix(y_test, poly_predict)
print(poly_confusion_matrix) 

In [ ]:
metrics.precision_score(y_test, poly_predict)

In [1]:
metrics.recall_score(y_test, poly_predict)

NameError: ignored

In [ ]:
plot_svm(poly_svc, X, y)

In [ ]:
rbf_svc = SVC(kernel = 'rbf', C=6, gamma = 2, random_state = 42)


# grid_search = GridSearchCV(rbf_svc, {'C':[0.2,0.5,1,2,3,4,5,6],'gamma':[2,4,6,8,10]}, cv=5, scoring ="recall") 
# grid_search.fit(crash_prepared, y_train)
# grid_search.best_params_ 
#The grid search found different values for C and gamma depending on if we want better recall or precision. We will use 3 and 8 respectively.


rbf_svc.fit(crash_prepared, y_train)
rbf_predict = rbf_svc.predict(crash_test_prepared)

rbf_confusion_matrix = metrics.confusion_matrix(y_test, rbf_predict)
print(rbf_confusion_matrix) 

# rbf_svc.fit(X_train, y_train)
# rbf_predict = rbf_svc.predict(X_test)
# rbf_confusion_matrix = metrics.confusion_matrix(y_test, rbf_predict)
# print(rbf_confusion_matrix) 
#we can see from the confusion matrix that there are minimal false positives and negatives.

[[18613    71]
 [ 1270    46]]


In [ ]:
grid_search.best_params_ 

{'C': 6, 'gamma': 2}

In [41]:
metrics.precision_score(y_test, rbf_predict)

0.39316239316239315

In [42]:
metrics.recall_score(y_test, rbf_predict)

0.034954407294832825